In [ ]:
import pickle
import numpy as np

import sys
import time
import threading
import logging
import inspect
import argparse
import os.path
import collections
import datetime
from datetime import date

sys.path.insert(0, "/Jts/tws api/source/pythonclient")

from ibapi import wrapper
from ibapi import utils
from ibapi.utils import iswrapper
from ibapi.execution import Execution
from ibapi.execution import ExecutionFilter
from ibapi.commission_report import CommissionReport
from ibapi.tag_value import TagValue
from ibapi.account_summary_tags import *

In [ ]:
'''
Change from ibapi to ohlc format
'''
def ibapi_to_ohlc(filename, mkt_hours = False):
    ohlc_per_symbol = {}
    
    data = pickle.load(open(filename, "rb"))
    
    for bar in data:
        ts = int(bar.date)
        ohlc_per_symbol[ts] = bar.close
    return ohlc_per_symbol

In [ ]:
'''
Fill in zeros where data does not exist into each dictionary of data per day
'''
def edit_data(day_dictionary):
    to_del = []
    
    for day in day_dictionary:
        count = 0
        num_entries = len(day_dictionary[day])
        
        for m in range(1, num_entries):
            if day_dictionary[day][m][3] == 0:
                count += 1
                day_dictionary[day][m][0:4] = day_dictionary[day][m - 1][3]
                
        for m in range(1, num_entries + 1):
            if day_dictionary[day][-m][3] == 0:
                count += 1
                day_dictionary[day][-m][0 : 4] = day_dictionary[day][-(m - 1)][3]
        
        if count > 100:
            to_del.append(day)
            
    for day in to_del:
        del day_dictionary[day]
    return day_dictionary

In [ ]:
'''
Generate data for all stocks in `symbols` array
'''
symbols = ['SPY', 'GME','TSLA','GOOG','AAPL']
all_data = {}
for sym in symbols:
    all_data[sym] = ibapi_to_ohlc("../raw_pricedata_{s}.pkl".format(s=sym))

In [ ]:
for sym in symbols:
    f = open("../processed_{:s}.csv".format(sym), "wt")
    csv_string = ""
    for ts in sorted(all_data[sym]):
        csv_string += str(ts) + ", " + str(date.fromtimestamp(ts)) + ", " + str(all_data[sym][ts]) + "\n"
    f.write(csv_string)
    f.close()
    
data = pickle.load(open("../raw_pricedata_SPY.pkl", "rb"))
all_data['DIA']